## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
# File location and type
file_location = "/FileStore/tables/GimeMeCredit.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)


In [3]:
# Create a view or table

temp_table_name = "GimeMeCredit_csv"

df.createOrReplaceTempView(temp_table_name)

In [4]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "GimeMeCredit_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [6]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)
labelIndexer.labels

Out[10]: ['0.0', '1.0']

In [7]:
plan_indexer = StringIndexer(inputCol = 'intl_plan', outputCol = 'intl_plan_indexed')

In [8]:
# Create the feature column
assembler = VectorAssembler(
  inputCols = ['Revolving', 'age', 'Num3059', 'DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumLoans', 'Num6089', 'NumberOfDependents'],
  outputCol = 'features')

In [9]:

# Training the model
classifier = RandomForestClassifier(labelCol = 'label', featuresCol = 'features')
pipeline = Pipeline(stages=[labelIndexer, assembler, classifier])
model = pipeline.fit(df)

In [10]:
# Evaluating the model
# Need to do that over a test set, not the same training set. 
from pyspark.ml.evaluation import BinaryClassificationEvaluator
 
predictions = model.transform(df)
evaluator = BinaryClassificationEvaluator()
auroc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

In [11]:
auroc



Out[17]: 0.8295433532567636